# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import numpy as np
import re 
import datetime as DT

In [2]:
df = pd.read_csv('main_task.csv')
df.sample(7)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
19289,id_6108,London,"['Japanese', 'American', 'Asian', 'Korean', 'B...",6117.0,5.0,$,7.0,"[['Awesome', 'Fabulous food'], ['12/02/2017', ...",/Restaurant_Review-g186338-d12539657-Reviews-C...,d12539657
3156,id_953,Hamburg,"['Asian', 'Indonesian', 'Halal']",955.0,4.0,$$ - $$$,30.0,"[['Hidden beauty', 'No good service and wrong ...",/Restaurant_Review-g187331-d8450304-Reviews-Ja...,d8450304
15590,id_6056,Barcelona,['European'],6057.0,4.0,NaN,26.0,"[[], []]",/Restaurant_Review-g187497-d4479230-Reviews-Fe...,d4479230
19927,id_1799,Madrid,NaN,1801.0,4.0,NaN,31.0,"[['Great choices & fair peices'], ['05/22/2017']]",/Restaurant_Review-g187514-d11278601-Reviews-L...,d11278601
14028,id_2635,Hamburg,['Indian'],2642.0,3.0,NaN,9.0,"[[], []]",/Restaurant_Review-g187331-d3747833-Reviews-Kr...,d3747833
33948,id_2779,Madrid,NaN,2781.0,4.5,NaN,18.0,"[[], []]",/Restaurant_Review-g187514-d5796131-Reviews-Re...,d5796131
24527,id_9094,Paris,['Indian'],9096.0,4.0,$,12.0,[['Mauritian-Indian subtlety in the food & fr....,/Restaurant_Review-g187147-d8015472-Reviews-Ma...,d8015472


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [4]:
df.describe(include = ['object'])

,Restaurant_id,City,Cuisine Style,Price Range,Reviews,URL_TA,ID_TA
count,40000,40000,30717,26114,40000,40000,40000
unique,11909,31,9007,3,33516,39980,39980
top,id_633,London,['Italian'],$$ - $$$,"[[], []]",/Restaurant_Review-g187514-d13002276-Reviews-D...,d9802625
freq,18,5757,1032,18412,6471,2,2


In [5]:
df['Price Range'].value_counts() #смотрим на столбец с данными о цене

$$ - $$$    18412
$            6279
$$$$         1423
Name: Price Range, dtype: int64

In [6]:
# заполняем пустые значения на возможные в рамказ вероятностного значения
nan_data = df['Price Range'].isna() # создайм переменную с нулевыми значениями 
# считаю вероятнасть того или иного значения
p = df['Price Range'].value_counts() / len(df['Price Range'].dropna())
# заполняем пропуски с вероятностью `p`
df.loc[nan_data, 'Price Range'] = \
    np.random.choice(p.index.to_list(), 
                      size=nan_data.sum(), 
                      p=p.to_list())
df['Price Range'].describe(include = ['object']) # проверяем полученные данные

count        40000
unique           3
top       $$ - $$$
freq         28237
Name: Price Range, dtype: object

In [7]:
def find_item(cell):
    if item in cell:
        return 1
    return 0

In [8]:
Price_Range = ('$', '$$ - $$$', '$$$$')
#добавляем цифровые столбцы для определения ценового диапозона ресторана
for item in Price_Range:
    df[item] = df['Price Range'].apply(find_item)

In [9]:
# заполняем пустые значения на возможные в рамказ вероятностного значения
nan_data = df['Cuisine Style'].isna() # создайм переменную с нулевыми значениями 
# считаю вероятнасть того или иного значения
p = df['Cuisine Style'].value_counts() / len(df['Cuisine Style'].dropna())
# заполняем пропуски с вероятностью `p`
df.loc[nan_data, 'Cuisine Style'] = \
    np.random.choice(p.index.to_list(), 
                      size=nan_data.sum(), 
                      p=p.to_list())
df['Cuisine Style'].describe(include = ['object']) # проверяем полученные данные

count           40000
unique           9007
top       ['Italian']
freq             1348
Name: Cuisine Style, dtype: object

In [10]:
df['Cuisine Style'] = df['Cuisine Style'].str.replace('[', '')
df['Cuisine Style'] = df['Cuisine Style'].str.replace(']', '')
df['Cuisine Style'] = df['Cuisine Style'].str.replace("'", '')
df['Cuisine Style'] = df['Cuisine Style'].str.replace('&', '')
df['Cuisine Style'] = df['Cuisine Style'].str.replace(', ', ',')
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda s: s.split(','))
df['Cuisine Style']

0                        [European, French, International]
1                                        [Japanese, Sushi]
2        [Japanese, Sushi, Asian, Grill, Vegetarian Fri...
3                              [Chinese, Minority Chinese]
4          [German, Central European, Vegetarian Friendly]
                               ...                        
39995    [Italian, Vegetarian Friendly, Vegan Options, ...
39996    [French, American, Bar, European, Vegetarian F...
39997                                    [Japanese, Sushi]
39998    [Polish, European, Eastern European, Central E...
39999                                            [Spanish]
Name: Cuisine Style, Length: 40000, dtype: object

In [11]:
#в названиях кухонь есть обощения, такие как Азиатская кухня или Средиземноморская, нужно их разбить на 
#названия кухонь конкретных стран, что бы улучшить прогноз
#сначала попробовал только на Азиатской кухне, МАЕ улучшился
Mediterranean = ['Italian', 'Greek', 'Spanish', 'French']
European = ['Albanian', 'Argentinean','Austrian', 'Belgian', 'British', 'Croatian', 'Czech', 'Danish',
            'Dutch',  'Norwegian', 'French', 'German', 'Greek', 'Hungarian', 'Irish', 'Italian', 
            'Portuguese', 'Polish', 'Romanian', 'Scottish', 'Slovenian', 'Spanish', 'Swedish', 
             'Welsh']
Arabic = ['Afghani', 'Egyptian', 'Lebanese', 'Moroccan', 'Pakistani', 'Persian', 'Tunisian']
Polynesian = ['Filipino', 'Fujian', 'Indonesian', 'Malaysian']
Scandinavian = ['Norwegian', 'Danish', 'Swedish']
African = ['Ethiopian', 'South American']
Latin = ['Argentinean', 'Brazilian', 'Chilean', 'Colombian', 'Cuban', 'Ecuadorean', 'Mexican',
         'Nepali', 'Peruvian', 'Salvadoran', 'Venezuelan']
Caribbean = ['Cajun  Creole', 'Jamaican', 'Cuban']
Caucasian = ['Armenian', 'Azerbaijani', 'Turkish', 'Georgian']
Southwestern = ['Burmese', 'Indonesian', 'Malaysian', 'Singaporean', 'Thai']
Eastern_European = ['Albanian', 'Croatian',  'Hungarian',  'Romanian', 'Russian', 'Slovenian',
                    'Polish', 'Ukrainian']
Central_European = ['Austrian', 'Belgian', 'Dutch', 'French',   'Czech',
                    'Swiss', 'German']
Middle_Eastern = ['Armenian', 'Georgian', 'Egyptian', 'Israeli', 'Lebanese', 'Turkish', 'Uzbek']
Asian = ['Central Asian', 'Chinese', 'Filipino', 'Indonesian', 'Japanese', 'Korean', 'Malaysian', 'Minority Chinese',
         'Singaporean','Sushi', 'Taiwanese', 'Thai', 'Vietnamese']

In [12]:
def Cuisine(cusion_list, cusions): #задаём функцию, что бы перебрать обобщающие кухни, такие как Азиатская кухня
    
    #перебираем списки кухонь
    for i in df['Cuisine Style']: 
        if len(i) == 1 and i == cusion_list:
            i = cusions
    return(i)

In [13]:
cusion_list = ['Mediterranean', 'European', 'Arabic', 'Polynesian', 'Scandinavian', 'African', 'Latin', 'Caribbean',
              'Caucasian', 'Asian', 'Southwestern', 'Eastern European', 'Central European', 'Middle Eastern']
cusions = [Mediterranean, European, Arabic, Polynesian, Scandinavian, African, Latin, Caribbean, Caucasian,
          Asian, Southwestern, Eastern_European, Central_European, Middle_Eastern]

In [14]:
for i in range(len(cusion_list)):
    Cuisine(cusion_list[i], cusions[i])

In [15]:
df['Cuisine Style']

0                        [European, French, International]
1                                        [Japanese, Sushi]
2        [Japanese, Sushi, Asian, Grill, Vegetarian Fri...
3                              [Chinese, Minority Chinese]
4          [German, Central European, Vegetarian Friendly]
                               ...                        
39995    [Italian, Vegetarian Friendly, Vegan Options, ...
39996    [French, American, Bar, European, Vegetarian F...
39997                                    [Japanese, Sushi]
39998    [Polish, European, Eastern European, Central E...
39999                                            [Spanish]
Name: Cuisine Style, Length: 40000, dtype: object

In [16]:
Cuisine_Style = set()  # создаём пустое множество для хранения уникальных значений кухонь
for style in df['Cuisine Style']:# начинаем перебор всех списков с кухнями
    for Cuisine in style: # начинаем перебор всех кухань из списков
        Cuisine_Style.add(Cuisine)

In [17]:
print(Cuisine_Style)

{'Polish', 'Belgian', 'Romanian', 'Dutch', 'Brazilian', 'Fujian', 'Hungarian', 'Central American', 'Diner', 'Jamaican', 'Filipino', 'Norwegian', 'Fast Food', 'Taiwanese', 'Pizza', 'Portuguese', 'New Zealand', 'Minority Chinese', 'Contemporary', 'Bangladeshi', 'Southwestern', 'Ukrainian', 'Cajun  Creole', 'Sri Lankan', 'Nepali', 'Pub', 'Native American', 'Cafe', 'French', 'Central Asian', 'Singaporean', 'German', 'Persian', 'Swiss', 'Caribbean', 'Swedish', 'Seafood', 'Venezuelan', 'Steakhouse', 'Wine Bar', 'Croatian', 'Bar', 'Peruvian', 'African', 'Tibetan', 'Welsh', 'Yunnan', 'Czech', 'Hawaiian', 'South American', 'Eastern European', 'European', 'Russian', 'Soups', 'Canadian', 'Gastropub', 'Halal', 'Arabic', 'Spanish', 'Australian', 'International', 'Georgian', 'Sushi', 'Middle Eastern', 'Austrian', 'Moroccan', 'Lebanese', 'Chilean', 'Barbecue', 'Chinese', 'Vegetarian Friendly', 'British', 'Tunisian', 'Israeli', 'Afghani', 'Street Food', 'Latin', 'Egyptian', 'Grill', 'Albanian', 'Colom

In [18]:
Cuisine_Style_count = {}  # создаём пустой словарь для хранения информации об кухнях
for style in df['Cuisine Style']:# начинаем перебор всех списков с кухнями
    for Cuisine in style: # начинаем перебор всех кухань из списков
        Cuisine_Style_count[Cuisine] = 0 # добавляем в словарь ключ, соответствующий очередной кухне
for style in df['Cuisine Style']:# начинаем перебор всех списков с кухнями
    for Cuisine in style: # начинаем перебор всех кухань из списков
        Cuisine_Style_count[Cuisine] += 1   # увеличиваем значение нужного ключа в словаре на 1

In [19]:
list_cuisine = list(Cuisine_Style_count.items())
list_cuisine.sort(key=lambda i: i[0])
print(list_cuisine)

[('Afghani', 29), ('African', 196), ('Albanian', 9), ('American', 1733), ('Arabic', 67), ('Argentinean', 223), ('Armenian', 15), ('Asian', 3896), ('Australian', 33), ('Austrian', 492), ('Azerbaijani', 2), ('Balti', 105), ('Bangladeshi', 100), ('Bar', 4242), ('Barbecue', 708), ('Belgian', 340), ('Brazilian', 167), ('Brew Pub', 303), ('British', 2105), ('Burmese', 2), ('Cafe', 3012), ('Cajun  Creole', 26), ('Cambodian', 23), ('Canadian', 6), ('Caribbean', 123), ('Caucasian', 5), ('Central American', 176), ('Central Asian', 17), ('Central European', 1824), ('Chilean', 9), ('Chinese', 1481), ('Colombian', 29), ('Contemporary', 679), ('Croatian', 38), ('Cuban', 31), ('Czech', 779), ('Danish', 239), ('Delicatessen', 502), ('Diner', 400), ('Dutch', 378), ('Eastern European', 664), ('Ecuadorean', 5), ('Egyptian', 25), ('Ethiopian', 53), ('European', 13051), ('Fast Food', 2213), ('Filipino', 13), ('French', 4188), ('Fujian', 2), ('Fusion', 750), ('Gastropub', 603), ('Georgian', 23), ('German', 

In [20]:
for item in Cuisine_Style:
    df[item] = df['Cuisine Style'].apply(find_item)
    
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: len(x))

In [21]:
df.sample(7)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,...,Balti,Greek,Burmese,Italian,Korean,Ecuadorean,Danish,Azerbaijani,Argentinean,Mongolian
32517,id_2070,Prague,2,2073.0,5.0,$$ - $$$,NaN,"[[], []]",/Restaurant_Review-g274707-d7124442-Reviews-Cu...,d7124442,...,0,0,0,0,0,0,0,0,0,0
16081,id_2075,Barcelona,2,2076.0,4.0,$$ - $$$,95.0,"[['Food and design', 'Quality, quality and qua...",/Restaurant_Review-g187497-d7171656-Reviews-To...,d7171656,...,0,0,0,0,0,0,0,0,0,0
18832,id_10710,London,2,10720.0,4.5,$$ - $$$,3.0,"[['Good quality food', 'If you love Lebanese f...",/Restaurant_Review-g186338-d6464554-Reviews-Sa...,d6464554,...,0,0,0,0,0,0,0,0,0,0
11930,id_66,Edinburgh,4,67.0,4.5,$$ - $$$,610.0,"[['👍❤️👍❤️👍❤️', 'Welcome to Meat Heaven!'], ['0...",/Restaurant_Review-g186525-d2159384-Reviews-Sh...,d2159384,...,0,0,0,0,0,0,0,0,0,0
30055,id_86,Berlin,6,87.0,4.5,$$ - $$$,271.0,"[['Nice', 'First date and beyond'], ['12/23/20...",/Restaurant_Review-g187323-d2014748-Reviews-Be...,d2014748,...,0,0,0,1,0,0,0,0,0,0
3282,id_4589,Berlin,1,4591.0,4.0,$,7.0,"[[], []]",/Restaurant_Review-g187323-d8786577-Reviews-Ta...,d8786577,...,0,0,0,0,0,0,0,0,0,0
31085,id_6924,Barcelona,2,6925.0,3.0,$,NaN,"[[], []]",/Restaurant_Review-g187497-d1081756-Reviews-Vi...,d1081756,...,0,0,0,0,0,0,0,0,0,0


In [22]:
pattern = re.compile("\d+\/\d+\/\d+") #находим все текстовые значения похожие на дату
df['date_rev'] = df['Reviews'].apply(pattern.findall) #создаём новый столбец только с датами
df['date_rev']

#отсеиваем случаи когда дата имеется в комментариямх
df['date_rev'] = df['date_rev'].apply(lambda x: [x[-2], x[-1]] if len(x)>=3 else x)
df['date_rev']

#случаи с обной датой и с двумя датами
df['rev_1'] = df['date_rev'].apply(lambda x: x[0] if len(x)>=1 else None)
df['rev_2'] = df['date_rev'].apply(lambda x: x[1] if len(x)==2 else None)
df['rev_2']

df['rev_1'] = pd.to_datetime(df['rev_1'])
df['rev_2'] = pd.to_datetime(df['rev_2'])

print('Самая свежая дата:', df[['rev_1', 'rev_2']].max().max())
print('Самая ранняя дата:', df[['rev_1','rev_2']].min().min())
max_days = abs(df['rev_1'] - df['rev_2']).max()
print('Количество дней:', max_days)

Самая свежая дата: 2018-02-26 00:00:00
Самая ранняя дата: 2004-04-21 00:00:00
Количество дней: 3207 days 00:00:00


In [23]:
df['date_rev_fresh'] = int(df[['rev_1', 'rev_2']].max().max().timestamp())
df['date_rev_fresh']

0        1519603200
1        1519603200
2        1519603200
3        1519603200
4        1519603200
            ...    
39995    1519603200
39996    1519603200
39997    1519603200
39998    1519603200
39999    1519603200
Name: date_rev_fresh, Length: 40000, dtype: int64

In [24]:
City = set()  # создаём пустое множество для хранения уникальных значений городов
for town in df['City']: # начинаем перебор городами
    City.add(town) # добавляем уникальные города   

In [25]:
for item in City:
    df[item] = df['City'].apply(find_item)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 173 entries, Restaurant_id to Athens
dtypes: datetime64[ns](2), float64(3), int64(161), object(7)
memory usage: 52.8+ MB


In [27]:
df.sample(7)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,...,Lyon,Berlin,Prague,Stockholm,Vienna,Hamburg,Oslo,Barcelona,Luxembourg,Athens
10698,id_4817,London,4,4826.0,4.0,$$ - $$$,191.0,"[['Quaint Pub, Good Food', 'Experience reviewe...",/Restaurant_Review-g186338-d2483661-Reviews-Th...,d2483661,...,0,0,0,0,0,0,0,0,0,0
9347,id_1867,Amsterdam,6,1872.0,3.5,$$ - $$$,121.0,"[['Nice place', 'Ok Breakfast'], ['11/17/2017'...",/Restaurant_Review-g188590-d8395743-Reviews-Ca...,d8395743,...,0,0,0,0,0,0,0,0,0,0
32766,id_3003,Lisbon,2,3006.0,4.0,$$$$,12.0,"[[], []]",/Restaurant_Review-g189158-d3177106-Reviews-An...,d3177106,...,0,0,0,0,0,0,0,0,0,0
38432,id_4673,London,3,4681.0,5.0,$,21.0,[['Best value brunch in Hackney and fantastic....,/Restaurant_Review-g186338-d8358727-Reviews-Pa...,d8358727,...,0,0,0,0,0,0,0,0,0,0
19520,id_1198,Athens,1,1200.0,4.5,$,9.0,"[['Bohemian, friendly, unique!', 'Vegan Rock-n...",/Restaurant_Review-g189400-d10215087-Reviews-5...,d10215087,...,0,0,0,0,0,0,0,0,0,1
37612,id_3432,Paris,1,3433.0,4.5,$$ - $$$,40.0,"[['Very good Chinese restaurant in Paris.', 'G...",/Restaurant_Review-g187147-d9468440-Reviews-JI...,d9468440,...,0,0,0,0,0,0,0,0,0,0
31882,id_858,Rome,6,859.0,4.5,$,218.0,"[['Loved the food', 'Delicious paninis'], ['11...",/Restaurant_Review-g187791-d7214382-Reviews-Ci...,d7214382,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [29]:
df.describe(include = ['object']) #смотрим какие столбцы остались не числовыми

,Restaurant_id,City,Price Range,Reviews,URL_TA,ID_TA,date_rev
count,40000,40000,40000,40000,40000,40000,40000
unique,11909,31,3,33516,39980,39980,24606
top,id_633,London,$$ - $$$,"[[], []]",/Restaurant_Review-g187514-d13002276-Reviews-D...,d9802625,[]
freq,18,5757,28237,6471,2,2,6471


In [30]:
#удаляем не числовые столбцы
df = df.drop(['Restaurant_id', 'City', 'Price Range', 'Reviews', 'URL_TA', 
              'ID_TA', 'date_rev', 'rev_1', 'rev_2'], axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 164 entries, Cuisine Style to Athens
dtypes: float64(3), int64(161)
memory usage: 50.0 MB


df = df.drop(['Mediterranean', 'European', 'Arabic', 'Polynesian', 'Scandinavian', 'African', 'Latin', 'Caribbean',
              'Caucasian', 'Asian', 'Southwestern', 'Eastern European', 'Central European', 'Middle Eastern'], axis = 1)
df.info()

In [31]:
df['Number of Reviews'] = df['Number of Reviews'].fillna(0) #заполняем значения NaN в столбце на нули

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [32]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Rating'], axis = 1)
y = df['Rating']

In [33]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [34]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [35]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [36]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [37]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21187199999999998
